In [1]:
from typing import Collection
import pandas as pd
import numpy as np
import gspread 
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from Naked.toolshed.shell import muterun_js, execute_js
from distutils.dir_util import copy_tree
import shutil
# import jsbeautifier
import os
import urllib.request
import requests
import json
import re
import sys
import hashlib
from datetime import datetime
import time

In [2]:
pd.options.display.html.use_mathjax = False

In [5]:
from create_dir import *
# from create_content import *
from pytexit import py2tex

In [15]:
py2tex("P_CgetsB")

<IPython.core.display.Latex object>

$$P_{CgetsB}$$


'$$P_{CgetsB}$$'